In [1]:
%load_ext autoreload
%autoreload 2
import torch
from torch import optim,nn
from torch.autograd import Variable
import numpy as np
import time
import os
os.chdir('..')
from models.data_loader import DataLoader

from models.retain_time import RETAIN

In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [3]:
# hyperparameters
epochs = 30
batch_size = 50
max_seq_length = 300
min_seq_length = 5
num_classes = 268
emb_size = 128
hid_size = 128
lr = 0.001
cuda_flag = True

# data loader
D = DataLoader(batch_size=batch_size,
   data_dir='data/batches/',
    mode='train', max_seq_length=max_seq_length, min_seq_length=min_seq_length)

model = RETAIN(emb_size,hid_size,num_classes,cuda_flag)
model.release = True

criterion = nn.CrossEntropyLoss()
cnt = 0
if cuda_flag:
    model.cuda()
    criterion.cuda()

In [6]:
# train model
lr_list = [0.0003, 0.0001, 0.00003, 0.00001, 0.000003, 0.000001]
lr_counter = 0
lr = lr_list[lr_counter]
opt = optim.Adam(model.parameters(), lr=lr)
loss_list = []
loss_mean = 0.0
file_cnt = 0
cnt = 0
loss_dict = dict()

In [ ]:
len_train = len(D.train_list)
while file_cnt<(epochs*len(D.train_list)):
    idx = file_cnt%(len_train)
    file = D.train_list[idx]
    if file not in loss_dict:
        loss_dict[file] = []
    print("Epoch %d [%d, %d/%d] - opening file %s" %(((file_cnt+1)/len_train), file_cnt, idx, len_train, D.train_list[idx]))
    file_num = int(file.split('_')[1].split('.')[0])
    D.batch_size = int(20000/file_num)
    D.load_batch_file(file)
    loss_list = []
    for i in range(D.batch_count):
        cnt+=1
#         input_list, targets = D.get_batch()
        input_list, targets, dates = D.get_batch()
        dates = Variable(torch.Tensor(dates), requires_grad=False)
        if cuda_flag:
            dates = dates.cuda()
        start = time.time()
        inputs = model.list_to_tensor(input_list)
        outputs = model(inputs,dates)
        targets = Variable(torch.LongTensor(targets)[:,-1]) # to only use last of each sequence
        if cuda_flag:
            targets = targets.cuda()
        loss = criterion(outputs.view(-1,num_classes),targets)
        loss_list.append(loss.data[0])
        if cnt%10==0:
            print('[%d] %1.3f' %(cnt,loss.data[0]))
        if cnt%500==0:
            print("Saving model at %dth step" %cnt)
            torch.save(model.state_dict(),'data/saved_weights/retain_time_%d.pth'%(cnt))
            # create CPU version
            model2 = RETAIN(emb_size,hid_size,num_classes,False)
            if cuda_flag:
                model.cpu()
            model2.load_state_dict(model.state_dict())
            torch.save(model2.state_dict(),'data/saved_weights/retain_time_%d_cpu.pth'%(cnt))
            del model2
            if cuda_flag:
                model.cuda()
            print("Saving at %dth step"%cnt)
        loss.backward()
        opt.step()
    print("Loss: %1.3f" %np.mean(loss_list))
    loss_dict[file].append(loss.data[0])
    file_cnt+=1

Epoch 1 [559, 149/410] - opening file 2014_22.pckl
[2010] 2.625
[2020] 2.525
Loss: 2.692
Epoch 1 [560, 150/410] - opening file 2014_76.pckl
Loss: 2.801
Epoch 1 [561, 151/410] - opening file 2014_113.pckl
Loss: 2.382
Epoch 1 [562, 152/410] - opening file 2014_160.pckl
Loss: 0.994
Epoch 1 [563, 153/410] - opening file 2014_275.pckl
Loss: 1.243
Epoch 1 [564, 154/410] - opening file 2015_100.pckl
Loss: 2.223
Epoch 1 [565, 155/410] - opening file 2015_75.pckl
[2030] 2.591
Loss: 2.579
Epoch 1 [566, 156/410] - opening file 2015_96.pckl
Loss: 2.845
Epoch 1 [567, 157/410] - opening file 2014_117.pckl
Loss: 2.652
Epoch 1 [568, 158/410] - opening file 2015_90.pckl
Loss: 2.743
Epoch 1 [569, 159/410] - opening file 2015_82.pckl
Loss: 2.754
Epoch 1 [570, 160/410] - opening file 2014_191.pckl
Loss: 1.646
Epoch 1 [571, 161/410] - opening file 2014_99.pckl
[2040] 2.835
Loss: 2.835
Epoch 1 [572, 162/410] - opening file 2014_227.pckl
Loss: 1.340
Epoch 1 [573, 163/410] - opening file 2015_25.pckl
[2050] 2

[2350] 0.326
Loss: 0.326
Epoch 1 [681, 271/410] - opening file 2014_276.pckl
Loss: 2.061
Epoch 1 [682, 272/410] - opening file 2014_69.pckl
Loss: 2.691
Epoch 1 [683, 273/410] - opening file 2015_43.pckl
[2360] 2.715
Loss: 2.736
Epoch 1 [684, 274/410] - opening file 2015_88.pckl
Loss: 3.075
Epoch 1 [685, 275/410] - opening file 2014_181.pckl
Loss: 1.632
Epoch 1 [686, 276/410] - opening file 2014_46.pckl
[2370] 2.706
Loss: 2.743
Epoch 1 [687, 277/410] - opening file 2014_265.pckl
Loss: 1.011
Epoch 1 [688, 278/410] - opening file 2014_40.pckl
[2380] 2.589
Loss: 2.699
Epoch 1 [689, 279/410] - opening file 2014_235.pckl
Loss: 1.339
Epoch 1 [690, 280/410] - opening file 2015_40.pckl
[2390] 2.676
Loss: 2.682
Epoch 1 [691, 281/410] - opening file 2014_71.pckl
[2400] 2.947
Loss: 2.897
Epoch 1 [692, 282/410] - opening file 2014_205.pckl
Loss: 1.499
Epoch 1 [693, 283/410] - opening file 2015_131.pckl
Loss: 2.441
Epoch 1 [694, 284/410] - opening file 2014_57.pckl
Loss: 2.665
Epoch 1 [695, 285/410]

Loss: 2.414
Epoch 1 [801, 391/410] - opening file 2015_282.pckl
[2750] 2.845
Loss: 2.845
Epoch 1 [802, 392/410] - opening file 2014_260.pckl
Loss: 2.385
Epoch 1 [803, 393/410] - opening file 2014_9.pckl
[2760] 2.747
[2770] 2.750
Loss: 2.790
Epoch 1 [804, 394/410] - opening file 2015_196.pckl
Loss: 1.568
Epoch 1 [805, 395/410] - opening file 2014_164.pckl
Loss: 0.911
Epoch 1 [806, 396/410] - opening file 2014_43.pckl
[2780] 2.773
Loss: 2.745
Epoch 1 [807, 397/410] - opening file 2014_24.pckl
[2790] 2.678
[2800] 2.805
Loss: 2.680
Epoch 1 [808, 398/410] - opening file 2015_284.pckl
Loss: 0.929
Epoch 1 [809, 399/410] - opening file 2015_168.pckl
Loss: 1.840
Epoch 1 [810, 400/410] - opening file 2015_242.pckl
Loss: 1.731
Epoch 1 [811, 401/410] - opening file 2014_259.pckl
Loss: 1.015
Epoch 1 [812, 402/410] - opening file 2015_36.pckl
[2810] 2.699
Loss: 2.734
Epoch 1 [813, 403/410] - opening file 2014_283.pckl
Loss: 0.745
Epoch 1 [814, 404/410] - opening file 2014_139.pckl
[2820] 2.510
Loss:

Loss: 2.563
Epoch 2 [921, 101/410] - opening file 2015_245.pckl
Loss: 0.468
Epoch 2 [922, 102/410] - opening file 2015_225.pckl
Loss: 1.196
Epoch 2 [923, 103/410] - opening file 2015_221.pckl
Loss: 1.628
Epoch 2 [924, 104/410] - opening file 2015_162.pckl
Loss: 1.077
Epoch 2 [925, 105/410] - opening file 2014_171.pckl
Loss: 1.766
Epoch 2 [926, 106/410] - opening file 2014_196.pckl
Loss: 1.939
Epoch 2 [927, 107/410] - opening file 2014_282.pckl
[3260] 0.000
Loss: 0.000
Epoch 2 [928, 108/410] - opening file 2014_74.pckl
Loss: 2.743
Epoch 2 [929, 109/410] - opening file 2014_141.pckl
Loss: 2.089
Epoch 2 [930, 110/410] - opening file 2015_232.pckl
Loss: 1.016
Epoch 2 [931, 111/410] - opening file 2014_169.pckl
Loss: 2.028
Epoch 2 [932, 112/410] - opening file 2015_109.pckl
Loss: 2.530
Epoch 2 [933, 113/410] - opening file 2015_17.pckl
[3270] 2.622
[3280] 2.702
[3290] 2.541
Loss: 2.649
Epoch 2 [934, 114/410] - opening file 2015_134.pckl
Loss: 2.479
Epoch 2 [935, 115/410] - opening file 2015

Loss: 1.678
Epoch 2 [1042, 222/410] - opening file 2015_63.pckl
[3600] 2.696
Loss: 2.744
Epoch 2 [1043, 223/410] - opening file 2014_77.pckl
Loss: 2.740
Epoch 2 [1044, 224/410] - opening file 2015_205.pckl
Loss: 1.422
Epoch 2 [1045, 225/410] - opening file 2015_73.pckl
Loss: 2.632
Epoch 2 [1046, 226/410] - opening file 2014_186.pckl
Loss: 1.385
Epoch 2 [1047, 227/410] - opening file 2015_150.pckl
Loss: 1.864
Epoch 2 [1048, 228/410] - opening file 2015_69.pckl
[3610] 2.769
Loss: 2.453
Epoch 2 [1049, 229/410] - opening file 2015_250.pckl
Loss: 1.571
Epoch 2 [1050, 230/410] - opening file 2014_15.pckl
[3620] 2.595
[3630] 2.604
Loss: 2.649
Epoch 2 [1051, 231/410] - opening file 2014_299.pckl
Loss: 0.003
Epoch 2 [1052, 232/410] - opening file 2014_118.pckl
[3640] 2.719
Loss: 2.719
Epoch 2 [1053, 233/410] - opening file 2014_162.pckl
Loss: 1.650
Epoch 2 [1054, 234/410] - opening file 2014_278.pckl
Loss: 0.863
Epoch 2 [1055, 235/410] - opening file 2014_106.pckl
Loss: 2.574
Epoch 2 [1056, 236

...

Loss: 2.484
Epoch 6 [2585, 125/410] - opening file 2015_164.pckl
Loss: 1.015
Epoch 6 [2586, 126/410] - opening file 2014_224.pckl
Loss: 1.817
Epoch 6 [2587, 127/410] - opening file 2015_30.pckl
[8960] 2.692
[8970] 2.638
Loss: 2.642
Epoch 6 [2588, 128/410] - opening file 2015_279.pckl
Loss: 0.428
Epoch 6 [2589, 129/410] - opening file 2015_258.pckl
Loss: 0.361
Epoch 6 [2590, 130/410] - opening file 2015_210.pckl
Loss: 0.753
Epoch 6 [2591, 131/410] - opening file 2015_269.pckl
Loss: 0.427
Epoch 6 [2592, 132/410] - opening file 2015_144.pckl
Loss: 1.921
Epoch 6 [2593, 133/410] - opening file 2015_231.pckl
Loss: 1.228
Epoch 6 [2594, 134/410] - opening file 2014_188.pckl
[8980] 0.965
Loss: 0.965
Epoch 6 [2595, 135/410] - opening file 2015_143.pckl
Loss: 2.307
Epoch 6 [2596, 136/410] - opening file 2014_173.pckl
Loss: 1.194
Epoch 6 [2597, 137/410] - opening file 2014_115.pckl
Loss: 2.351
Epoch 6 [2598, 138/410] - opening file 2014_56.pckl
Loss: 2.625
Epoch 6 [2599, 139/410] - opening file 20

Loss: 0.659
Epoch 6 [2704, 244/410] - opening file 2015_107.pckl
Loss: 2.357
Epoch 6 [2705, 245/410] - opening file 2015_249.pckl
Loss: 0.554
Epoch 6 [2706, 246/410] - opening file 2015_180.pckl
Loss: 0.929
Epoch 6 [2707, 247/410] - opening file 2015_111.pckl
Loss: 2.649
Epoch 6 [2708, 248/410] - opening file 2015_67.pckl
[9330] 2.269
Loss: 2.569
Epoch 6 [2709, 249/410] - opening file 2015_113.pckl
Loss: 2.449
Epoch 6 [2710, 250/410] - opening file 2014_218.pckl
Loss: 0.309
Epoch 6 [2711, 251/410] - opening file 2014_82.pckl
Loss: 2.603
Epoch 6 [2712, 252/410] - opening file 2015_198.pckl
Loss: 1.012
Epoch 6 [2713, 253/410] - opening file 2015_159.pckl
Loss: 1.113
Epoch 6 [2714, 254/410] - opening file 2015_71.pckl
Loss: 2.648
Epoch 6 [2715, 255/410] - opening file 2015_87.pckl
[9340] 2.576
Loss: 2.542
Epoch 6 [2716, 256/410] - opening file 2014_179.pckl
Loss: 1.666
Epoch 6 [2717, 257/410] - opening file 2015_64.pckl
Loss: 2.589
Epoch 6 [2718, 258/410] - opening file 2014_92.pckl
Loss:

Loss: 2.550
Epoch 6 [2822, 362/410] - opening file 2014_6.pckl
[9750] 2.921
[9760] 2.911
Loss: 2.905
Epoch 6 [2823, 363/410] - opening file 2014_207.pckl
Loss: 1.009
Epoch 6 [2824, 364/410] - opening file 2015_114.pckl
Loss: 2.111
Epoch 6 [2825, 365/410] - opening file 2014_270.pckl
Loss: 0.317
Epoch 6 [2826, 366/410] - opening file 2015_132.pckl
Loss: 2.022
Epoch 6 [2827, 367/410] - opening file 2015_218.pckl
[9770] 1.073
Loss: 1.073
Epoch 6 [2828, 368/410] - opening file 2014_52.pckl
Loss: 2.673
Epoch 6 [2829, 369/410] - opening file 2014_220.pckl
Loss: 0.486
Epoch 6 [2830, 370/410] - opening file 2015_78.pckl
[9780] 2.704
Loss: 2.685
Epoch 6 [2831, 371/410] - opening file 2014_177.pckl
Loss: 1.067
Epoch 6 [2832, 372/410] - opening file 2014_267.pckl
Loss: 0.852
Epoch 6 [2833, 373/410] - opening file 2015_86.pckl
Loss: 2.701
Epoch 6 [2834, 374/410] - opening file 2014_295.pckl
Loss: 0.341
Epoch 6 [2835, 375/410] - opening file 2015_201.pckl
Loss: 0.844
Epoch 6 [2836, 376/410] - openi

Loss: 2.375
Epoch 7 [2941, 71/410] - opening file 2015_176.pckl
Loss: 1.478
Epoch 7 [2942, 72/410] - opening file 2015_151.pckl
Loss: 1.196
Epoch 7 [2943, 73/410] - opening file 2015_268.pckl
Loss: 0.057
Epoch 7 [2944, 74/410] - opening file 2014_121.pckl
Loss: 2.347
Epoch 7 [2945, 75/410] - opening file 2014_284.pckl
Loss: 0.201
Epoch 7 [2946, 76/410] - opening file 2014_19.pckl
[10150] 2.619
[10160] 2.630
Loss: 2.614
Epoch 7 [2947, 77/410] - opening file 2014_36.pckl
[10170] 2.760
Loss: 2.649
Epoch 7 [2948, 78/410] - opening file 2015_179.pckl
[10180] 1.866
Loss: 1.866
Epoch 7 [2949, 79/410] - opening file 2015_47.pckl
Loss: 2.675
Epoch 7 [2950, 80/410] - opening file 2015_295.pckl
Loss: 0.177
Epoch 7 [2951, 81/410] - opening file 2015_5.pckl
[10190] 2.958
[10200] 2.993
Loss: 3.004
Epoch 7 [2952, 82/410] - opening file 2015_46.pckl
[10210] 2.630
Loss: 2.594
Epoch 7 [2953, 83/410] - opening file 2015_26.pckl
[10220] 2.655
[10230] 2.646
Loss: 2.636
Epoch 7 [2954, 84/410] - opening file

Loss: 1.791
Epoch 7 [3058, 188/410] - opening file 2014_201.pckl
Loss: 1.024
Epoch 7 [3059, 189/410] - opening file 2014_189.pckl
Loss: 1.338
Epoch 7 [3060, 190/410] - opening file 2015_241.pckl
Loss: 0.159
Epoch 7 [3061, 191/410] - opening file 2015_133.pckl
Loss: 2.116
Epoch 7 [3062, 192/410] - opening file 2015_44.pckl
[10600] 2.695
Loss: 2.627
Epoch 7 [3063, 193/410] - opening file 2015_122.pckl
Loss: 2.507
Epoch 7 [3064, 194/410] - opening file 2014_286.pckl
Loss: 0.052
Epoch 7 [3065, 195/410] - opening file 2014_119.pckl
Loss: 2.088
Epoch 7 [3066, 196/410] - opening file 2015_166.pckl
Loss: 1.177
Epoch 7 [3067, 197/410] - opening file 2014_210.pckl
Loss: 0.312
Epoch 7 [3068, 198/410] - opening file 2015_155.pckl
[10610] 1.050
Loss: 1.050
Epoch 7 [3069, 199/410] - opening file 2015_188.pckl
Loss: 1.220
Epoch 7 [3070, 200/410] - opening file 2015_167.pckl
Loss: 1.168
Epoch 7 [3071, 201/410] - opening file 2014_178.pckl
Loss: 1.135
Epoch 7 [3072, 202/410] - opening file 2015_247.pck

In [8]:
# lower learning rate
lr_counter+=1
lr = lr_list[lr_counter]
opt = optim.Adam(model.parameters(), lr=lr)
print(lr)

3e-05


In [4]:
# load new model
cnt = 2000
model.load_state_dict(torch.load('/home/mjc/github/EHRVis/data/saved_weights/retain_time_%d.pth'%cnt))
model.train()
file_cnt = 559